In [22]:
import psycopg2, psycopg2.extras, psycopg2.pool, pickle, json, matplotlib, os, sys, shapely
from multiprocessing import Pool, Manager; from collections import OrderedDict; from shapely.geometry import mapping, shape; from sqlalchemy import create_engine
import matplotlib.pyplot as plt; import numpy as np; import pandas as pd; import geopandas as gpd; import seaborn as sns
%matplotlib inline

# Latest Tile Statistics

This notebook queries the `latest-tile-data-v3` database to identify tiles with specific characteristics, such as:
- named road km
- total road km
- total buildings
- number of buildings with more than `building=yes`

This allows for further _intrinsic quality_ comparison between the histories of tiles that are currently quantitatively similar on the map.

In [23]:
#Create the engine for Pandas to query SQL with
engine = create_engine('postgresql://anderstj@127.0.0.1:5432/latest-tile-data-v3')

In [24]:
study_tiles = json.loads(open('study_tiles.json','r').read())
print("Found {0} tiles.".format(len(study_tiles)))

Found 7 tiles.


In [25]:
# Get study_tile stats
def get_tile_df(quadkey):
    q_s = """SELECT * FROM roads, buildings, geometry WHERE 
        roads.quadkey = buildings.quadkey AND 
        geometry.quadkey = roads.quadkey AND 
        roads.quadkey = '{0}'""".format(quadkey)
    df = pd.read_sql_query(q_s,con=engine)
    
    df['named_road_ratio'] = (df.named_edited_km + df.named_new_km) / df.total_km
    df['more_building_ratio'] = (df.new_buildings_more + df.edited_buildings_more) / df.total_buildings
    return df

for tile in study_tiles:
    print(tile['name'], end="...")
    tile['characteristics'] = get_tile_df(tile['quad'])

Port Au Prince, Haiti...Tacloban, Philippines...Monrovia, Liberia...Trisuli Bazar, Nepal...Heidelberg, Germany...London, UK...Manhattan, NY...

In [26]:
def get_similar_tiles(tile, tolerance=0.01, ratio_tol=0.01):
    """Given a tile, find similar tiles in the _current database_ within specific thresholds."""
    df = tile['characteristics']
    min_roads = (df.total_km - df.total_km*tolerance).values[0]
    max_roads = (df.total_km + df.total_km*tolerance).values[0]
    min_buildings = (df.total_buildings - df.total_buildings*tolerance).values[0]
    max_buildings = (df.total_buildings + df.total_buildings*tolerance).values[0]
    min_roads_ratio = (df.named_road_ratio - df.named_road_ratio*ratio_tol).values[0]
    max_roads_ratio = (df.named_road_ratio + df.named_road_ratio*ratio_tol).values[0]    
    min_building_ratio = (df.more_building_ratio - df.more_building_ratio*ratio_tol).values[0]
    max_building_ratio = (df.more_building_ratio + df.more_building_ratio*ratio_tol).values[0]

    query_string = """SELECT * FROM roads, buildings, geometry WHERE roads.quadkey = buildings.quadkey AND geometry.quadkey = roads.quadkey AND 
    roads.total_km > {0} AND roads.total_km < {1} AND roads.quadkey != '{4}' AND 
    buildings.total_buildings > {2} AND buildings.total_buildings < {3} AND
    (roads.named_edited_km + roads.named_new_km)/roads.total_km > {5} AND 
    (roads.named_edited_km + roads.named_new_km)/roads.total_km < {6} AND
    (buildings.edited_buildings_more + buildings.new_buildings_more)::float / buildings.total_buildings::float > {7} AND
    (buildings.edited_buildings_more + buildings.new_buildings_more)::float / buildings.total_buildings::float < {8}
    """.format(min_roads, max_roads, min_buildings, max_buildings, df.quadkey.values[0][0], min_roads_ratio, max_roads_ratio, min_building_ratio, max_building_ratio)
    return pd.read_sql_query(query_string,con=engine)

# Test approach
x = get_similar_tiles(study_tiles[2], tolerance = 0.25, ratio_tol=0.25)
print(len(x))
x.head()

0


,quadkey,named_edited_km,named_new_km,total_edited_km,total_km,total_new_km,unnamed_edited_km,unnamed_new_km,quadkey,total_buildings,total_new_buildings,total_edited_buildings,new_buildings_more,new_buildings_yes,edited_buildings_more,edited_buildings_yes,quadkey,coordinates,type


## Identify Similar Tiles in the Database

For each of the study tiles, how many other tiles exist with similar characteristics: 
 - Number of Buildings
 - km of roads
 - % of named roads
 - % of buildings with more than just `building==yes`

In [27]:
sim_tiles = get_similar_tiles(study_tiles[0], tolerance=0.15, ratio_tol=0.05)
sim_tiles

,quadkey,named_edited_km,named_new_km,total_edited_km,total_km,total_new_km,unnamed_edited_km,unnamed_new_km,quadkey,total_buildings,total_new_buildings,total_edited_buildings,new_buildings_more,new_buildings_yes,edited_buildings_more,edited_buildings_yes,quadkey,coordinates,type
0,033110210131,453.059,87.4549,675.205,956.69,281.486,222.146,194.031,033110210131,10745,8533,2212,513,8020,258,1954,033110210131,"[[[-9.228515625, 38.61687046392974], [-9.22851...",Polygon


In [28]:
for tile in study_tiles:
    print(tile['name'])
    sim_tiles = get_similar_tiles(tile, tolerance=0.25, ratio_tol=0.35)
    print("Similar Tiles: {0}\n".format(len(sim_tiles)))
    tile['similar_tiles'] = sim_tiles

Port Au Prince, Haiti
Similar Tiles: 8

Tacloban, Philippines
Similar Tiles: 1

Monrovia, Liberia
Similar Tiles: 1

Trisuli Bazar, Nepal
Similar Tiles: 4

Heidelberg, Germany
Similar Tiles: 37

London, UK
Similar Tiles: 17

Manhattan, NY
Similar Tiles: 8



In [29]:
#Override Haiti & Nepal
study_tiles[0]['similar_tiles'] = get_similar_tiles(study_tiles[0], tolerance=0.15, ratio_tol=0.05)
study_tiles[3]['similar_tiles'] = get_similar_tiles(study_tiles[3], tolerance=0.20, ratio_tol=0.15)

In [30]:
for tile in study_tiles:
    print(tile['name'])
    print("Similar Tiles: {0}\n".format(len(tile['similar_tiles'])))

Port Au Prince, Haiti
Similar Tiles: 1

Tacloban, Philippines
Similar Tiles: 1

Monrovia, Liberia
Similar Tiles: 1

Trisuli Bazar, Nepal
Similar Tiles: 1

Heidelberg, Germany
Similar Tiles: 37

London, UK
Similar Tiles: 17

Manhattan, NY
Similar Tiles: 8



In [31]:
study_tiles[3]['similar_tiles']

,quadkey,named_edited_km,named_new_km,total_edited_km,total_km,total_new_km,unnamed_edited_km,unnamed_new_km,quadkey,total_buildings,total_new_buildings,total_edited_buildings,new_buildings_more,new_buildings_yes,edited_buildings_more,edited_buildings_yes,quadkey,coordinates,type
0,123131220022,8.50502,0.0,198.51,262.506,63.996,190.005,63.996,123131220022,6237,5696,541,854,4842,55,486,123131220022,"[[[84.375, 27.994401411046148], [84.375, 28.07...",Polygon


In [32]:
def convert_to_gpd(df):
    """ Given a pandas dataframe with a `geometry` column, return a GeoDataFrame
        Note: This function creates the geometry from JSON string and then gets the envelope, this ensures proper winding order.
    """
    df['geometry'] = df.coordinates.apply(lambda coords: shape({"type":"Polygon", "coordinates":json.loads(coords)}).exterior.envelope)
    
    return gpd.GeoDataFrame(df)
y = convert_to_gpd(x)

### Write GeoJSON feature collections of similar tiles for each study tile

In [33]:
directory = '/data/www/jennings/iscram/'
if not os.path.exists(directory):
    os.makedirs(directory)
for tile in study_tiles:
    print(tile['name'])
    tile_gpd = convert_to_gpd(tile['similar_tiles'])
    filename = directory+"/"+tile['name']+"_sim_tiles.geojson"
    if os.path.exists(filename):
        os.remove(filename)
    try:
        tile_gpd.to_file(directory+"/"+tile['name']+"_sim_tiles.geojson", driver="GeoJSON")
    except:
        print("\tError. Length of dataframe: {0}".format(len(tile_gpd)))

Port Au Prince, Haiti
Tacloban, Philippines
Monrovia, Liberia
Trisuli Bazar, Nepal
Heidelberg, Germany
London, UK
Manhattan, NY


### Create GeoJSON file for all features

In [34]:
feat_coll = {"type":"FeatureCollection","features":[]}
for tile in study_tiles:
    tile['characteristics']['name'] = tile['name']
    feat_coll['features'].append( json.loads(convert_to_gpd(tile['characteristics']).to_json())['features'][0] )

with open(directory+"study_tiles.geojson",'w') as oFile:
    json.dump(feat_coll, oFile)

### The study tiles can be [visualized here](http://www.townsendjennings.com/geojson-polygons?geojson=http://epic-analytics.cs.colorado.edu:9000/jennings/iscram/study_tiles.geojson#1.9/14.6/8.5)
(Must be on the CU network)

## Create Single JSON record with study tiles and comparable tiles

In [35]:
output = []
for tile in study_tiles: 
    sim_quads = [x[0] for x in tile['similar_tiles'].quadkey.values]
    output.append(
        {'name':tile['name'],
         'quad':tile['quad'],
         'compare_tiles': [{'quad':x} for x in sim_quads]})
with open("study_tiles_with_compare_tiles.json",'w') as oFile:
    json.dump(output,oFile)

In [36]:
print(study_tiles[3]['name'])
study_tiles[3]['characteristics']

Trisuli Bazar, Nepal


,quadkey,named_edited_km,named_new_km,total_edited_km,total_km,total_new_km,unnamed_edited_km,unnamed_new_km,quadkey,total_buildings,...,new_buildings_yes,edited_buildings_more,edited_buildings_yes,quadkey,coordinates,type,named_road_ratio,more_building_ratio,name,geometry
0,123131221200,10.1151,0.318314,242.562,324.699,82.1367,232.447,81.8184,123131221200,7596,...,5848,137,498,123131221200,"[[[85.078125, 27.91676664124907], [85.078125, ...",Polygon,0.032133,0.16456,"Trisuli Bazar, Nepal","POLYGON ((85.078125 27.91676664124907, 85.1660..."


In [50]:
from IPython import display
for x in study_tiles[:4]: 
    display.display((x['name']))
    df = x['characteristics']

    df['total_more_buildings']   = df.edited_buildings_more+df.new_buildings_more
    df['more_buildings_percent'] = df.total_more_buildings / df.total_buildings * 100

    df['total_named_km']   = df.named_edited_km+df.named_new_km
    df['named_km_percent'] = df.total_named_km / df.total_km * 100
    
    display.display(df[['total_buildings','more_buildings_percent','total_km','named_km_percent']])
    
    compare_df = x['similar_tiles']
    compare_df['total_more_buildings']   = compare_df.edited_buildings_more+compare_df.new_buildings_more
    compare_df['more_buildings_percent'] = compare_df.total_more_buildings / compare_df.total_buildings * 100
    compare_df['total_named_km']   = compare_df.named_edited_km+compare_df.named_new_km
    compare_df['named_km_percent'] = compare_df.total_named_km / compare_df.total_km * 100
    
    display.display(compare_df[['quadkey','total_buildings','more_buildings_percent','total_km','named_km_percent']])

'Port Au Prince, Haiti'

,total_buildings,more_buildings_percent,total_km,named_km_percent
0,12141,6.869286,1006.15,54.01143


,quadkey,quadkey,quadkey,total_buildings,more_buildings_percent,total_km,named_km_percent
0,033110210131,033110210131,033110210131,10745,7.17543,956.69,56.498333


'Tacloban, Philippines'

,total_buildings,more_buildings_percent,total_km,named_km_percent
0,29573,71.068204,257.339,35.431847


...

,total_buildings,more_buildings_percent,total_km,named_km_percent
0,19193,5.569739,174.068,32.192861


,quadkey,quadkey,quadkey,total_buildings,more_buildings_percent,total_km,named_km_percent
0,310101233020,310101233020,310101233020,17210,3.689715,146.896,30.022097


'Trisuli Bazar, Nepal'

,total_buildings,more_buildings_percent,total_km,named_km_percent
0,7596,16.456029,324.699,3.213257


,quadkey,quadkey,quadkey,total_buildings,more_buildings_percent,total_km,named_km_percent
0,123131220022,123131220022,123131220022,6237,14.574315,262.506,3.239934


In [21]:
for tile in study_tiles:
    print(tile['name'])
    display.display(tile['quad'])
    display.display(tile['similar_tiles'])

Port Au Prince, Haiti


'032211203001'

,quadkey,named_edited_km,named_new_km,total_edited_km,total_km,total_new_km,unnamed_edited_km,unnamed_new_km,quadkey,total_buildings,total_new_buildings,total_edited_buildings,new_buildings_more,new_buildings_yes,edited_buildings_more,edited_buildings_yes,quadkey,coordinates,type,geometry
0,033110210131,453.059,87.4549,675.205,956.69,281.486,222.146,194.031,033110210131,10745,8533,2212,513,8020,258,1954,033110210131,"[[[-9.228515625, 38.61687046392974], [-9.22851...",Polygon,"POLYGON ((-9.228515625 38.61687046392974, -9.1..."


Tacloban, Philippines


'132312223332'

...

'031313131103'

,quadkey,named_edited_km,named_new_km,total_edited_km,total_km,total_new_km,unnamed_edited_km,unnamed_new_km,quadkey,total_buildings,total_new_buildings,total_edited_buildings,new_buildings_more,new_buildings_yes,edited_buildings_more,edited_buildings_yes,quadkey,coordinates,type,geometry
0,120212011000,315.388,30.46680,575.410,758.365,182.9540,260.022,152.4880,120212011000,21775,7722,14053,1497,6225,5707,8346,120212011000,"[[[13.359375, 52.429222277955134], [13.359375,...",Polygon,"POLYGON ((13.359375 52.42922227795513, 13.4472..."
1,120210232333,420.123,23.07970,717.234,875.303,158.0690,297.111,134.9900,120210232333,16856,4176,12680,1219,2957,5590,7090,120210232333,"[[[13.271484375, 52.48278022207821], [13.27148...",Polygon,"POLYGON ((13.271484375 52.48278022207821, 13.3..."
2,120203101113,344.671,28.36530,664.466,810.513,146.0480,319.794,117.6830,120203101113,14846,6307,8539,1028,5279,3205,5334,120203101113,"[[[9.755859375, 52.3755991766591], [9.75585937...",Polygon,"POLYGON ((9.755859375 52.3755991766591, 9.8437..."
3,120212011001,335.563,29.41770,636.140,797.916,161.7760,300.577,132.3590,120212011001,17195,7457,9738,744,6713,4175,5563,120212011001,"[[[13.447265625, 52.429222277955134], [13.4472...",Polygon,"POLYGON ((13.447265625 52.42922227795513, 13.5..."
4,031313131121,496.248,60.20570,638.314,808.342,170.0280,142.067,109.8220,031313131121,14780,11597,3183,3795,7802,927,2256,031313131121,"[[[-0.263671875, 51.45400691005982], [-0.26367...",Polygon,"POLYGON ((-0.263671875 51.45400691005982, -0.1..."
5,120203030003,256.082,40.72130,442.297,619.346,177.0490,186.215,136.3280,120203030003,22706,10933,11773,3691,7242,4530,7243,120203030003,"[[[7.119140625, 51.50874245880334], [7.1191406...",Polygon,"POLYGON ((7.119140625 51.50874245880334, 7.207..."
6,120202132303,330.968,32.39340,596.115,757.512,161.3970,265.147,129.0030,120202132303,19277,11661,7616,4373,7288,3091,4525,120202132303,"[[[4.658203125, 50.84757295365388], [4.6582031...",Polygon,"POLYGON ((4.658203125 50.84757295365388, 4.746..."
7,120203213123,254.234,32.42810,477.309,629.231,151.9220,223.076,119.4930,120203213123,15402,10642,4760,2847,7795,1986,2774,120203213123,"[[[8.173828125, 50.06419173665909], [8.1738281...",Polygon,"POLYGON ((8.173828125 50.06419173665909, 8.261..."
8,120320201300,333.281,25.24650,514.262,588.183,73.9211,180.981,48.6746,120320201300,21678,2416,19262,875,1541,7798,11464,120320201300,"[[[23.5546875, 46.739860599692676], [23.554687...",Polygon,"POLYGON ((23.5546875 46.73986059969268, 23.642..."
9,120230223013,319.785,66.85360,449.000,649.903,200.9030,129.215,134.0500,120230223013,20055,16691,3364,6023,10668,846,2518,120230223013,"[[[12.216796875, 45.460130637921004], [12.2167...",Polygon,"POLYGON ((12.216796875 45.460130637921, 12.304..."


Manhattan, NY


'032010110132'

,quadkey,named_edited_km,named_new_km,total_edited_km,total_km,total_new_km,unnamed_edited_km,unnamed_new_km,quadkey,total_buildings,total_new_buildings,total_edited_buildings,new_buildings_more,new_buildings_yes,edited_buildings_more,edited_buildings_yes,quadkey,coordinates,type,geometry
0,120220011030,565.542,91.4660,800.055,1047.010,246.960,234.513,155.4940,120220011030,58787,40613,18174,67,40546,1097,17077,120220011030,"[[[2.28515625, 48.748945343432936], [2.2851562...",Polygon,"POLYGON ((2.28515625 48.74894534343294, 2.3730..."
1,031331123312,481.431,64.1852,662.900,860.620,197.720,181.469,133.5340,031331123312,57156,21612,35544,141,21471,1788,33756,031331123312,"[[[-1.58203125, 47.1598400130443], [-1.5820312...",Polygon,"POLYGON ((-1.58203125 47.1598400130443, -1.494..."
2,120220330313,606.398,55.0455,793.246,993.775,200.529,186.848,145.4840,120220330313,44680,20950,23730,158,20792,755,22975,120220330313,"[[[4.833984375, 45.70617928533084], [4.8339843...",Polygon,"POLYGON ((4.833984375 45.70617928533084, 4.921..."
3,021212222131,522.204,62.3348,878.178,1020.460,142.286,355.974,79.9515,021212222131,58618,57586,1032,1904,55682,304,728,021212222131,"[[[-123.134765625, 49.2104204456503], [-123.13...",Polygon,"POLYGON ((-123.134765625 49.2104204456503, -12..."
4,030222231030,635.584,128.0370,870.206,1300.220,430.019,234.622,301.9820,030222231030,52353,47998,4355,171,47827,1008,3347,030222231030,"[[[-87.71484375, 41.83682786072713], [-87.7148...",Polygon,"POLYGON ((-87.71484375 41.83682786072713, -87...."
5,032010033002,564.971,41.8386,768.219,1029.000,260.782,203.248,218.9430,032010033002,55169,52778,2391,1148,51630,818,1573,032010033002,"[[[-76.640625, 39.23225314171488], [-76.640625...",Polygon,"POLYGON ((-76.640625 39.23225314171488, -76.55..."
6,023010230020,578.495,104.4920,714.426,994.779,280.353,135.931,175.8610,023010230020,40883,36149,4734,578,35571,485,4249,023010230020,"[[[-122.34375, 37.50972584293751], [-122.34375...",Polygon,"POLYGON ((-122.34375 37.50972584293751, -122.2..."
7,032300212020,450.859,269.1170,614.039,1001.500,387.457,163.180,118.3410,032300212020,53359,50551,2808,190,50361,1171,1637,032300212020,"[[[-66.09375, 18.396230138028827], [-66.09375,...",Polygon,"POLYGON ((-66.09375 18.39623013802883, -66.005..."
